In [1]:
# Importing modules
import pandas as pd
import os
import sys

# Read data into papers
#papers = pd.read_csv(os.path.join(sys.path[0],'../Dataset/ClinicalSTS2018/ClinicalSTS/clinicalSTS.train.txt'))
ClinicalTexts = pd.read_csv(os.path.join(sys.path[0],'data/clinicalSTS.train.txt'), sep='\t', header=None)

# Print head
ClinicalTexts.head()

,0,1,2
0,Insulin NPH Human [NOVOLIN N] 100 unit/mL susp...,Insulin NPH Human [NOVOLIN N] 100 unit/mL sus...,3.50
1,"Patient arrives ambulatory, Gait steady, Hist...","Complex assessment performed, Patient arrives...",2.50
2,"Peripheral IV site, established in the right ...","Peripheral IV site, present prior to arrival,...",3.45
3,No: new confusion or inability to stay alert ...,No: new confusion or inability to stay alert ...,4.00
4,Spent 15 minutes with the patient and greater ...,"Nurse visit ten minutes, over half of which w...",3.00


['Insulin NPH Human [NOVOLIN N] 100 unit/mL suspension subcutaneous as directed by prescriber.',
 ' Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears comfortable, Patient cooperative, alert, Skin warm.',
 ' Peripheral IV site, established in the right forearm, using an 18 gauge catheter, in one attempt.',
 ' No: new confusion or inability to stay alert and awake; currently struggling to breathe, even while inactive or resting; currently feeling like you are going to collapse every time you stand (sit); vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools; heavy vaginal bleeding or purple or red rash/blotches that stay when pressed by a glass (purpural rash)',
 'Spent 15 minutes with the patient and greater than 50% of this time was spent counseling the patient regarding diagnosis and available treatment options.']

## Import Libraries

In [3]:
### 1. Import Libraries

# for text preprocessing
import re
import spacy

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

# import numpy for matrix operation
import numpy as np

# Importing Gensim
import gensim
from gensim import corpora

# to suppress warnings
from warnings import filterwarnings
filterwarnings('ignore')

nlp = spacy.load('en_core_web_sm')

In [4]:
# to suppress warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [5]:
nlp = spacy.load('en_core_web_sm')

In [6]:
# combining all the documents into a list:

corpus = ClinicalTexts[0].tolist()

In [7]:
# the complete corpus as below:

corpus[0:5]

['Insulin NPH Human [NOVOLIN N] 100 unit/mL suspension subcutaneous as directed by prescriber.',
 ' Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears comfortable, Patient cooperative, alert, Skin warm.',
 ' Peripheral IV site, established in the right forearm, using an 18 gauge catheter, in one attempt.',
 ' No: new confusion or inability to stay alert and awake; currently struggling to breathe, even while inactive or resting; currently feeling like you are going to collapse every time you stand (sit); vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools; heavy vaginal bleeding or purple or red rash/blotches that stay when pressed by a glass (purpural rash)',
 'Spent 15 minutes with the patient and greater than 50% of this time was spent counseling the patient regarding diagnosis and available treatment options.']

## 2. Text Preprocessing

### Steps to preprocess text data:



    1. Convert the text into lowercase
    2. Split text into words
    3. Remove the stop loss words
    4. Remove the Punctuation, any symbols and special characters
    5. Normalize the word (I'll be using Lemmatization for normalization)



In [8]:
# Apply Preprocessing on the Corpus

# stop loss words 
stop = set(stopwords.words('english'))

# punctuation 
exclude = set(string.punctuation) 

# lemmatization
lemma = WordNetLemmatizer() 

# One function for all the steps:
def clean(doc):
    
    # convert text into lower case + split into words
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    
    # remove any stop words present
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)  
    
    # remove punctuations + normalize the text
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())  
    return normalized

# clean data stored in a new list
clean_corpus = [clean(doc).split() for doc in corpus]   

In [9]:
clean_corpus[0:5]

[['insulin',
  'nph',
  'human',
  'novolin',
  'n',
  '100',
  'unitml',
  'suspension',
  'subcutaneous',
  'directed',
  'prescriber'],
 ['patient',
  'arrives',
  'ambulatory',
  'gait',
  'steady',
  'history',
  'obtained',
  'patient',
  'patient',
  'appears',
  'comfortable',
  'patient',
  'cooperative',
  'alert',
  'skin',
  'warm'],
 ['peripheral',
  'iv',
  'site',
  'established',
  'right',
  'forearm',
  'using',
  '18',
  'gauge',
  'catheter',
  'one',
  'attempt'],
 ['no',
  'new',
  'confusion',
  'inability',
  'stay',
  'alert',
  'awake',
  'currently',
  'struggling',
  'breathe',
  'even',
  'inactive',
  'resting',
  'currently',
  'feeling',
  'like',
  'going',
  'collapse',
  'every',
  'time',
  'stand',
  'sit',
  'vomit',
  'look',
  'like',
  'ground',
  'coffee',
  'vomiting',
  'blood',
  'uncontrollable',
  'continuous',
  'rectal',
  'bleeding',
  'black',
  'sticky',
  'tarlike',
  'stool',
  'heavy',
  'vaginal',
  'bleeding',
  'purple',
  'red'

### 3. Creating Document Term Matrix

Using gensim for Document Term Matrix(DTM), we don't need to create the DTM matrix from scratch explicitly. The gensim library has internal mechanism to create the DTM.

The only requirement for gensis package is we need to pass the cleaned data in the form of tokenized words.

In [10]:
# Creating the term dictionary of our courpus that is of all the words (Sepcific to Genism syntax perspective), 
# where every unique term is assigned an index. 

dict_ = corpora.Dictionary(clean_corpus)

print(dict_)

Dictionary(1705 unique tokens: ['100', 'directed', 'human', 'insulin', 'n']...)


Now, the next step is to convert the corpus (the list of documents) into a document-term Matrix using the dictionary that we had prepared above. (The vectorizer used here is the Bag of Words).	

In [11]:
# Converting list of documents (corpus) into Document Term Matrix using the dictionary 
doc_term_matrix = [dict_.doc2bow(i) for i in clean_corpus]
doc_term_matrix[0:2]

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1)],
 [(11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 4),
  (21, 1),
  (22, 1),
  (23, 1)]]

The output implies: 

* Document wise we have the index of the word and its frequency.
* The 0th word is repeated 1 time, then the 1st word repeated 1 and so on ...	

### 4. Implementation of LDA

In [12]:
# Creating the object for LDA model using gensim library

Lda = gensim.models.ldamodel.LdaModel

In [13]:
# Running and Training LDA model on the document term matrix.

ldamodel = Lda(doc_term_matrix, num_topics=6, id2word = dict_, passes=1, random_state=0, eval_every=None)

In [14]:
# Prints the topics with the indexes: 0,1,2 :

ldamodel.print_topics()


[(0,
  '0.058*"patient" + 0.011*"discussed" + 0.011*"time" + 0.010*"member" + 0.010*"alert" + 0.009*"person" + 0.009*"understanding" + 0.009*"procedure" + 0.009*"risk" + 0.009*"stay"'),
 (1,
  '0.021*"pain" + 0.020*"no" + 0.020*"stay" + 0.018*"new" + 0.018*"inability" + 0.013*"awake" + 0.012*"alert" + 0.012*"confusion" + 0.011*"fever" + 0.010*"neck"'),
 (2,
  '0.019*"patient" + 0.011*"procedure" + 0.011*"discussed" + 0.010*"time" + 0.010*"one" + 0.010*"1" + 0.009*"tablet" + 0.008*"team" + 0.008*"member" + 0.008*"necessity"'),
 (3,
  '0.066*"learning" + 0.055*"patient" + 0.046*"identified" + 0.039*"barrier" + 0.032*"instruction" + 0.031*"used" + 0.031*"teaching" + 0.028*"understanding" + 0.024*"need" + 0.018*"included"'),
 (4,
  '0.082*"patient" + 0.013*"time" + 0.012*"history" + 0.010*"obtained" + 0.010*"person" + 0.010*"alert" + 0.010*"arrives" + 0.010*"appears" + 0.009*"place" + 0.009*"cooperative"'),
 (5,
  '0.024*"patient" + 0.014*"treatment" + 0.010*"risk" + 0.010*"consent" + 0.00

### 4a. Extracting Topics from the Corpus 

In [15]:
print(ldamodel.print_topics(num_topics=6, num_words=5))

# num_topics mean: how many topics want to extract 
# num_words: the number of words that want per topic

[(0, '0.058*"patient" + 0.011*"discussed" + 0.011*"time" + 0.010*"member" + 0.010*"alert"'), (1, '0.021*"pain" + 0.020*"no" + 0.020*"stay" + 0.018*"new" + 0.018*"inability"'), (2, '0.019*"patient" + 0.011*"procedure" + 0.011*"discussed" + 0.010*"time" + 0.010*"one"'), (3, '0.066*"learning" + 0.055*"patient" + 0.046*"identified" + 0.039*"barrier" + 0.032*"instruction"'), (4, '0.082*"patient" + 0.013*"time" + 0.012*"history" + 0.010*"obtained" + 0.010*"person"'), (5, '0.024*"patient" + 0.014*"treatment" + 0.010*"risk" + 0.010*"consent" + 0.009*"discussed"')]


** **
#### Step 6: Analyzing our LDA model <a class="anchor\" id="results"></a>
** **

Now that we have a trained model let’s visualize the topics for interpretability. To do so, we’ll use a popular visualization package, pyLDAvis which is designed to help interactively with:

1. Better understanding and interpreting individual topics, and
2. Better understanding the relationships between the topics.

For (1), you can manually select each topic to view its top most frequent and/or “relevant” terms, using different values of the λ parameter. This can help when you’re trying to assign a human interpretable name or “meaning” to each topic.

For (2), exploring the Intertopic Distance Plot can help you learn about how topics relate to each other, including potential higher-level structure between groups of topics.

In [16]:
num_topics=6

import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join(sys.path[0],'results/ldavis_med_'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dict_)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

#pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
pyLDAvis.save_html(LDAvis_prepared, os.path.join(sys.path[0],'results/ldavis_med_'+str(num_topics)+'.html'))

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.213689  0.087874       1        1  28.438775
3      0.215348  0.113638       2        1  25.066964
4      0.069629 -0.140835       3        1  18.014836
0     -0.005724  0.022159       4        1  13.303007
2     -0.088435 -0.055105       5        1   7.793225
5      0.022871 -0.027731       6        1   7.383193, topic_info=            Term        Freq       Total Category  logprob  loglift
207     learning  230.000000  230.000000  Default  30.0000  30.0000
20       patient  527.000000  527.000000  Default  29.0000  29.0000
201   identified  156.000000  156.000000  Default  28.0000  28.0000
212     teaching   97.000000   97.000000  Default  27.0000  27.0000
213         used  102.000000  102.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
476  information    3.908316   17.844954   Topic6  -5.4759   1.0873
58            no    5.525887   98.876474   Topic6  -5.1295  -0.2785
163         plan    4.635057   63.680258   Topic6  -5.3053  -0.0143
204  instruction    4.971151  138.827198   Topic6  -5.2353  -0.7236
526      symptom    4.101190   30.080741   Topic6  -5.4277   0.6134

[410 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
1093      3  0.776051    05
176       1  0.126396     1
176       2  0.094797     1
176       3  0.347590     1
176       4  0.094797     1
...     ...       ...   ...
541       1  0.273638  work
541       3  0.136819  work
541       6  0.547275  work
878       2  0.232832  year
878       3  0.698496  year

[831 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 5, 1, 3, 6])

### 4b. Assigning the topics to the documents

Pass the doc_term_matrix to the LDA model:

In [17]:
# printing the topic associations with the documents
count = 0
for i in ldamodel[doc_term_matrix]:
    print("doc : ",count,i)
    count += 1

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


doc :  0 [(0, 0.9303933), (1, 0.013891941), (2, 0.013975726), (3, 0.0139045315), (4, 0.0139036365), (5, 0.01393083)]
doc :  1 [(4, 0.95075077)]
doc :  2 [(0, 0.01283023), (1, 0.013093133), (2, 0.012942813), (3, 0.0128265545), (4, 0.93548566), (5, 0.012821566)]
doc :  3 [(0, 0.29120052), (1, 0.69515294)]
doc :  4 [(0, 0.010508719), (1, 0.010469542), (2, 0.010491648), (3, 0.010470713), (4, 0.010544311), (5, 0.9475151)]
doc :  5 [(1, 0.9560715)]
doc :  6 [(0, 0.26961976), (1, 0.71673226)]
doc :  7 [(0, 0.94429106), (1, 0.011164155), (2, 0.01111537), (3, 0.011145804), (4, 0.0111513315), (5, 0.011132287)]
doc :  8 [(0, 0.016850227), (1, 0.016721629), (2, 0.016880924), (3, 0.016780995), (4, 0.016742805), (5, 0.9160234)]
doc :  9 [(0, 0.015195352), (1, 0.015407887), (2, 0.015199277), (3, 0.92376983), (4, 0.015175752), (5, 0.015251851)]
doc :  10 [(0, 0.92383885), (1, 0.015207013), (2, 0.015266353), (3, 0.015226902), (4, 0.015247292), (5, 0.015213531)]
doc :  11 [(0, 0.021133354), (1, 0.020895

doc :  543 [(0, 0.013981822), (1, 0.013894959), (2, 0.01391099), (3, 0.93020135), (4, 0.014052007), (5, 0.013958821)]
doc :  544 [(0, 0.01395829), (1, 0.01389256), (2, 0.013905318), (3, 0.93038696), (4, 0.013955752), (5, 0.013901087)]
doc :  545 [(0, 0.77080196), (3, 0.1920544)]
doc :  546 [(0, 0.012917603), (1, 0.012838037), (2, 0.012834649), (3, 0.21288586), (4, 0.0130063705), (5, 0.7355175)]
doc :  547 [(0, 0.012874172), (1, 0.48845395), (2, 0.012933177), (3, 0.012830171), (4, 0.012832735), (5, 0.4600758)]
doc :  548 [(4, 0.9678198)]
doc :  549 [(0, 0.56319916), (3, 0.4110759)]
doc :  550 [(0, 0.940279), (1, 0.011992033), (2, 0.011927838), (3, 0.011921504), (4, 0.011950312), (5, 0.011929301)]
doc :  551 [(0, 0.01853339), (1, 0.018533368), (2, 0.018573254), (3, 0.9072773), (4, 0.018560171), (5, 0.018522533)]
doc :  552 [(4, 0.9650516)]
doc :  553 [(0, 0.0119732665), (1, 0.012042596), (2, 0.011941512), (3, 0.011965655), (4, 0.6470212), (5, 0.3050558)]
doc :  554 [(0, 0.0139262825), (1

In [18]:
# printing the topic associations with the documents
import operator
lstTopicsCorpus =[]

count = 0
for i in ldamodel[doc_term_matrix]:
    #print("doc : ",count, i)
    maxTopic = max(i,key=operator.itemgetter(1))[0]
    lstTopicsCorpus.append(maxTopic)
    count += 1

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
lstTopicsCorpus[0:5]

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[0, 4, 4, 1, 5]

**In this section we are merging the text corpus with the topic. So all the text rows topic will be tied up in one dataset**

In [20]:
df_Topic = pd.DataFrame(
    {'Text': corpus,
     'Cleaned_Text': clean_corpus,
     'Topic': lstTopicsCorpus
    })

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
pd.set_option('display.max_colwidth', None)

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
df_Topic

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Text,Cleaned_Text,Topic
0,Insulin NPH Human [NOVOLIN N] 100 unit/mL suspension subcutaneous as directed by prescriber.,"[insulin, nph, human, novolin, n, 100, unitml, suspension, subcutaneous, directed, prescriber]",0
1,"Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears comfortable, Patient cooperative, alert, Skin warm.","[patient, arrives, ambulatory, gait, steady, history, obtained, patient, patient, appears, comfortable, patient, cooperative, alert, skin, warm]",4
2,"Peripheral IV site, established in the right forearm, using an 18 gauge catheter, in one attempt.","[peripheral, iv, site, established, right, forearm, using, 18, gauge, catheter, one, attempt]",4
3,"No: new confusion or inability to stay alert and awake; currently struggling to breathe, even while inactive or resting; currently feeling like you are going to collapse every time you stand (sit); vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools; heavy vaginal bleeding or purple or red rash/blotches that stay when pressed by a glass (purpural rash)","[no, new, confusion, inability, stay, alert, awake, currently, struggling, breathe, even, inactive, resting, currently, feeling, like, going, collapse, every, time, stand, sit, vomit, look, like, ground, coffee, vomiting, blood, uncontrollable, continuous, rectal, bleeding, black, sticky, tarlike, stool, heavy, vaginal, bleeding, purple, red, rashblotches, stay, pressed, glass, purpural, rash]",1
4,Spent 15 minutes with the patient and greater than 50% of this time was spent counseling the patient regarding diagnosis and available treatment options.,"[spent, 15, minute, patient, greater, 50, time, spent, counseling, patient, regarding, diagnosis, available, treatment, option]",5
...,...,...,...
745,Discussed the necessity of other members of the team participating in the procedure with the patient.,"[discussed, necessity, member, team, participating, procedure, patient]",0
746,"Barriers to learning: (cultural, religious/spiritual, motivational, physical/cognitive, language, emotional); no apparent barriers to learning present, ready to learn.","[barrier, learning, cultural, religiousspiritual, motivational, physicalcognitive, language, emotional, apparent, barrier, learning, present, ready, learn]",3
747,"Unable to assess if there is a history or concern, or exposure to physical, emotional, sexual, financial abuse, neglect or domestic violence, Comments: family present.","[unable, ass, history, concern, exposure, physical, emotional, sexual, financial, abuse, neglect, domestic, violence, comment, family, present]",2
748,"Discussed the risks, benefits, alternatives, advance directives, and the necessity of other members of the healthcare team participating in the procedure.","[discussed, risk, benefit, alternative, advance, directive, necessity, member, healthcare, team, participating, procedure]",2


### Topics are tagged with the text

We can filter the texts according to the topics 

In [23]:
df_Topic.loc[df_Topic['Topic'] == 1]

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Text,Cleaned_Text,Topic
3,"No: new confusion or inability to stay alert and awake; currently struggling to breathe, even while inactive or resting; currently feeling like you are going to collapse every time you stand (sit); vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools; heavy vaginal bleeding or purple or red rash/blotches that stay when pressed by a glass (purpural rash)","[no, new, confusion, inability, stay, alert, awake, currently, struggling, breathe, even, inactive, resting, currently, feeling, like, going, collapse, every, time, stand, sit, vomit, look, like, ground, coffee, vomiting, blood, uncontrollable, continuous, rectal, bleeding, black, sticky, tarlike, stool, heavy, vaginal, bleeding, purple, red, rashblotches, stay, pressed, glass, purpural, rash]",1
5,No: new confusion or inability to stay alert and awake; newly painful neck with difficulty bending the neck; paralyzed face muscles or severe dizziness,"[no, new, confusion, inability, stay, alert, awake, newly, painful, neck, difficulty, bending, neck, paralyzed, face, muscle, severe, dizziness]",1
6,"No: new confusion or inability to stay alert and awake; struggling to breathe even while inactive or resting; currently feeling like you are going to collapse every time you stand or sit up; vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools; heavy vaginal bleeding or purple or red rash/blotches that stay when pressed by a glass (purpuric rash)","[no, new, confusion, inability, stay, alert, awake, struggling, breathe, even, inactive, resting, currently, feeling, like, going, collapse, every, time, stand, sit, up, vomit, look, like, ground, coffee, vomiting, blood, uncontrollable, continuous, rectal, bleeding, black, sticky, tarlike, stool, heavy, vaginal, bleeding, purple, red, rashblotches, stay, pressed, glass, purpuric, rash]",1
12,"No: new confusion or inability to stay alert and awake; currently feeling like you are going to collapse every time you stand (sit); complete inability to swallow; newly painful neck with difficulty bending the neck; purple or red rash/blotches that stay when pressed by a glass (purpural rash) or pain, pressure or tightness in the chest, jaw or arm","[no, new, confusion, inability, stay, alert, awake, currently, feeling, like, going, collapse, every, time, stand, sit, complete, inability, swallow, newly, painful, neck, difficulty, bending, neck, purple, red, rashblotches, stay, pressed, glass, purpural, rash, pain, pressure, tightness, chest, jaw, arm]",1
20,"They do not have a copy of the dates at this time, Asked that a copy of the immunization dates be brought to the next clinic appointment.","[copy, date, time, asked, copy, immunization, date, brought, next, clinic, appointment]",1
...,...,...,...
725,"Green or yellow discharge in the eye or on eyelashes continuing throughout the day, bloodshot or pink eyes","[green, yellow, discharge, eye, eyelash, continuing, throughout, day, bloodshot, pink, eye]",1
728,"Side rails up, Cart/Stretcher in lowest position, Call light within reach, Hospital ID band on, Patient safety, dignity, sense of well being and individual rights are respected.","[side, rail, up, cartstretcher, lowest, position, call, light, within, reach, hospital, id, band, on, patient, safety, dignity, sense, well, individual, right, respected]",1
735,"No: new shortness of breath at rest; pain, pressure or tightness unrelated to coughing in chest, jaw, or arm.","[no, new, shortness, breath, rest, pain, pressure, tightness, unrelated, coughing, chest, jaw, arm]",1
736,"No: new confusion or inability to stay alert and awake; struggling to breathe even while inactive or resting; any chest pain or discomfort; change in heart rate or rhythm or heart palpitations; excessive sweating; new weakness, numbness or tingling on one half of the body (an arm and leg) or curre

## Sentence Similarity Check

We will pick one sentence from one topic and check similarity among all the sentences

Try to find out where the test sentence is more similar

In [24]:
from sentence_transformers import SentenceTransformer

model_p = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
df_Topic['Text'].tolist()[0]

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'Insulin NPH Human [NOVOLIN N] 100 unit/mL suspension subcutaneous as directed by prescriber.'

In [26]:
# Getting the embedding
q_emb = model_p.encode(df_Topic['Text'].tolist()[0])
p_emb = model_p.encode(df_Topic['Text'].tolist())

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
from sentence_transformers import  util

# Printing the similarity score 
#print("Similarity on bigger:", util.pytorch_cos_sim(q_emb, p_emb))
sim_score = util.pytorch_cos_sim(q_emb, p_emb).cpu().detach().numpy()[0]

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
df_Score_0 = pd.DataFrame(
    {'Sim': sim_score,
     'Topic': lstTopicsCorpus
    })

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
df_Score_0

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Sim,Topic
0,1.000000,0
1,0.320613,4
2,0.345812,4
3,0.128784,1
4,0.257640,5
...,...,...
745,0.236219,0
746,0.025585,3
747,0.105145,2
748,0.140756,2


In [30]:
#df_Score_0.to_excel("output.xlsx")  

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
df_Score_0.groupby(['Topic']).agg({"Sim": ['mean','sum','min','max','count']})

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Sim                                     
           mean        sum       min       max count
Topic                                               
0      0.203963  24.271616 -0.025669  1.000000   119
1      0.124738  17.213879 -0.024156  0.395832   138
2      0.239498  17.004391 -0.014177  0.504735    71
3      0.192401  39.249855 -0.023461  0.431170   204
4      0.225056  33.533390  0.008919  0.394726   149
5      0.175966  12.141623 -0.059912  0.471832    69

## Create a function to display the statistics

In [32]:
def getSimiTopic(ind,p_emb):
    # Getting the embedding
    q_emb = model_p.encode(df_Topic['Text'].tolist()[ind])
    #p_emb = model_p.encode(df_Topic['Text'].tolist())
    
    print(df_Topic['Text'].tolist()[ind])
    
    # Printing the similarity score 
    sim_score = util.pytorch_cos_sim(q_emb, p_emb).cpu().detach().numpy()[0]
    
    # Data frame create
    df_Score = pd.DataFrame(
    {'Sim': sim_score,
     'Topic': lstTopicsCorpus
    })
    
    #display(df_Score.groupby(['Topic']).agg({"Sim": ['mean','sum','min','max','count']}))
    return df_Score.groupby(['Topic']).agg({"Sim": ['mean','sum','min','max','count']})

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
# topic 0
getSimiTopic(0, p_emb)

Insulin NPH Human [NOVOLIN N] 100 unit/mL suspension subcutaneous as directed by prescriber.


/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Sim                                     
           mean        sum       min       max count
Topic                                               
0      0.203963  24.271616 -0.025669  1.000000   119
1      0.124738  17.213879 -0.024156  0.395832   138
2      0.239498  17.004391 -0.014177  0.504735    71
3      0.192401  39.249855 -0.023461  0.431170   204
4      0.225056  33.533390  0.008919  0.394726   149
5      0.175966  12.141623 -0.059912  0.471832    69

In [34]:
# topic 1
df1 = getSimiTopic(3, p_emb)

 No: new confusion or inability to stay alert and awake; currently struggling to breathe, even while inactive or resting; currently feeling like you are going to collapse every time you stand (sit); vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools; heavy vaginal bleeding or purple or red rash/blotches that stay when pressed by a glass (purpural rash)


/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
# topic 1
df2 = getSimiTopic(5, p_emb)

No: new confusion or inability to stay alert and awake; newly painful neck with difficulty bending the neck; paralyzed face muscles or severe dizziness


/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
# topic 1
df3 = getSimiTopic(725, p_emb)

 Green or yellow discharge in the eye or on eyelashes continuing throughout the day, bloodshot or pink eyes


/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Trying to average on the 3 Topic 1 dataframes

In [37]:
pd.concat([df1, df2,df3]).groupby(level=0).mean()

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Sim                                     
           mean        sum       min       max count
Topic                                               
0      0.170289  20.264425 -0.025362  0.483620   119
1      0.407448  56.227814 -0.026985  1.000000   138
2      0.158378  11.244839  0.009804  0.469743    71
3      0.183935  37.522690 -0.048694  0.401400   204
4      0.183384  27.324274  0.024738  0.442322   149
5      0.187002  12.903113 -0.040399  0.566656    69

In [38]:
# topic 2
getSimiTopic(11,p_emb)

 The risks and benefits of the procedure were discussed, and the patient consented to this procedure.


/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Sim                                     
           mean        sum       min       max count
Topic                                               
0      0.477955  56.876686  0.000728  0.772398   119
1      0.207522  28.637970  0.010035  0.559080   138
2      0.372562  26.451920  0.069113  1.000001    71
3      0.369218  75.320427  0.115699  0.726581   204
4      0.369360  55.034630  0.037115  0.632715   149
5      0.404575  27.915695  0.035402  0.800146    69

In [39]:
# topic 3
getSimiTopic(9, p_emb)

 Negative gastrointestinal review of systems, Historian denies abdominal pain, nausea, vomiting.


/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Sim                                     
           mean        sum       min       max count
Topic                                               
0      0.181039  21.543676  0.031676  0.355950   119
1      0.253581  34.994114 -0.069916  0.608988   138
2      0.161750  11.484249 -0.024569  0.323135    71
3      0.220262  44.933407 -0.053316  1.000000   204
4      0.208497  31.065996 -0.001758  0.413647   149
5      0.196033  13.526261 -0.013015  0.628727    69

In [40]:
# topic 4
getSimiTopic(1,  p_emb)

 Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears comfortable, Patient cooperative, alert, Skin warm.


/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Sim                                     
           mean        sum       min       max count
Topic                                               
0      0.483993  57.595207  0.134863  0.930046   119
1      0.280693  38.735653  0.049835  0.597209   138
2      0.353945  25.130114  0.134603  0.524843    71
3      0.447255  91.240028  0.127826  0.733945   204
4      0.543260  80.945709  0.061611  1.000000   149
5      0.400787  27.654284  0.176410  0.615653    69

In [41]:
# topic 5
getSimiTopic(4,  p_emb)

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Spent 15 minutes with the patient and greater than 50% of this time was spent counseling the patient regarding diagnosis and available treatment options.


Sim                                     
           mean        sum       min       max count
Topic                                               
0      0.435203  51.789116  0.038055  0.726409   119
1      0.217102  29.960117  0.028898  0.474428   138
2      0.354929  25.199989  0.122470  0.708877    71
3      0.449296  91.656387  0.038516  0.715046   204
4      0.407817  60.764687  0.036580  0.615760   149
5      0.389876  26.901470  0.100088  1.000000    69

## Get a topic wise function

Instead of taking just a single sentence for checking against other cases, 


we can take all the cases of a topic and find the average similarity. This will give us a topic wise similarity score

In [42]:

def getTopicScore(topic, p_emb):
    # We need a list of dataframes for each topic 
    lst_df  = []
    
    # Get the list of index in the dataframe for the selected topic
    lst_ind = df_Topic.loc[df_Topic['Topic'] == topic].index.tolist()
    
    for ind in lst_ind:
        print("added", ind)
        df = getSimiTopic(ind,p_emb)
        lst_df.append(df)
    return pd.concat(lst_df).groupby(level=0).mean()


/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic 0 Simiarity Score

In [43]:
getTopicScore(0,p_emb)

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


added 0
Insulin NPH Human [NOVOLIN N] 100 unit/mL suspension subcutaneous as directed by prescriber.
added 7
 Cardiovascular assessment findings include heart rate normal, Heart rhythm, atrial fibrillation with controlled ventricular response.
added 10
 Given current medication regimen, the following parameters should be monitored by outpatient providers: None
added 13
 Discussed goals, risks, alternatives, advanced directives, and the necessity of other members of the surgical team participating in the procedure with the patient.
added 16
 Dimension 3 Emotional, Behavioral or Cognitive Conditions and Complications: The patient received a Risk Score of 1. Interference with Addiction Recovery Emotional concerns related to negative consequences and effects of addiction; patient is able to view these as part of addiction and recovery.
added 19
 Discussed risks, goals, and necessity of other members of the team participating in the procedure with patient.
added 26
Benefits and risks of acu

 Discussed the risks, the goals, the alternatives, advance directives, and the necessity of other members of the health care team who would be participating in the procedures with the patient.
added 338
 Patient discharged to home, ambulating without assistance, family driving, accompanied by parent, Discharge instructions given to mother.
added 344
 Discussed the risks, goals, alternatives, and advanced directives and the necessity of other members of the healthcare team participating in the procedure with the patient (or legal representative and other present during the discussion).
added 353
Individual is able to understand limited conversations about routine daily activities with familiar communication partners.
added 361
 Patient discharged to home, ambulating without assistance, family driving, accompanied by other family member, Discharge instructions given to patient.
added 368
 Skin assessment findings include skin warm, dry, normal in color, Inspection findings include lacera

BENEFITS AND RISKS OF ACUPUNCTURE WERE DISCUSSED WITH PATIENT AND PATIENT CONSENTS TO PROCEED WITH TREATMENT.
added 637
 After explanation of the test and answering the patient's questions, the patient verbalized an understanding.
added 641
The DCS fellow is available for questions from the primary service overnight through the hospital operator.
added 644
Patient tolerated treatment well and was able to demonstrate independence in performing the program.
added 647
 Patient requires extensive assistance in the following activities:  bathing, dressing, grooming/hygiene, toileting, transfer to/from bed/chair, mobility.
added 660
 Patient discharged to home, ambulating without assistance, family driving, accompanied by other family member, Above person(s) verbalized understanding of discharge instructions and follow-up care.
added 665
 Discussed the risks, goals, alternatives, and advanced directives and the necessity of other members of the healthcare team participating in the procedure 

Sim                                     
           mean        sum       min       max count
Topic                                               
0      0.426141  50.710831  0.038965  1.000000   119
1      0.194976  26.906700 -0.007990  0.503144   138
2      0.307324  21.820038  0.061607  0.627126    71
3      0.377580  77.026352  0.039483  0.702821   204
4      0.362129  53.957180  0.022210  0.607394   149
5      0.341318  23.550949  0.074974  0.672977    69

```
Sim
	mean 	sum 	min 	max 	count
Topic 					
0 	0.426141 	50.710831 	0.038965 	1.000000 	119
1 	0.194976 	26.906700 	-0.007990 	0.503144 	138
2 	0.307324 	21.820040 	0.061607 	0.627126 	71
3 	0.377580 	77.026360 	0.039483 	0.702821 	204
4 	0.362129 	53.957184 	0.022210 	0.607394 	149
5 	0.341318 	23.550951 	0.074974 	0.672977 	69

```

Output of the average of all the 119 sentences of Topic 0 compared with all the cases **Goes** with the hypothesis

The average is .42 which is highest 

In [44]:
getTopicScore(1,p_emb)

added 3
 No: new confusion or inability to stay alert and awake; currently struggling to breathe, even while inactive or resting; currently feeling like you are going to collapse every time you stand (sit); vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools; heavy vaginal bleeding or purple or red rash/blotches that stay when pressed by a glass (purpural rash)
added 5
No: new confusion or inability to stay alert and awake; newly painful neck with difficulty bending the neck; paralyzed face muscles or severe dizziness
added 6
 No: new confusion or inability to stay alert and awake; struggling to breathe even while inactive or resting; currently feeling like you are going to collapse every time you stand or sit up; vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools; heavy vaginal bleeding or purple or red rash/blotches that stay whe

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


 No: new shortness of breath; pain, pressure or tightness in the chest, jaw or arm or new confusion or inability to stay alert and awake
added 67
 No: new confusion or inability to stay alert and awake; currently struggling to breathe, even while inactive or resting or currently feeling like you are going to collapse every time you stand (sit)
added 72
 No: newly stiff or painful neck; purple or red rash/blotches that stay when pressed by a glass (purpural rash); currently feeling like you are going to collapse every time you stand (sit); complete inability to swallow or currently struggling to breathe, even while inactive or resting
added 82
 Following denial of allergies and review of potential side effects and complications, including, but not necessarily limited to, infection, allergic reaction, local tissue breakdown, and seizure, the patient indicated they understood and agreed to proceed.
added 85
 No: typical 'cold' symptoms; fever present (greater than or equal to 100.4 F or 3

No: fever each day for more than 72 hours; fever that has recurred after 24 hours without fever or temperature of 103?F ( 39.4?C) or greater (in an adult)
added 286
 No: new confusion or inability to stay alert and awake; currently feeling like you are going to collapse every time you stand (sit); newly stiff or painful neck; purple or red rash/blotches that stay when pressed by a glass (purpural rash) or pain, pressure or tightness in the chest, jaw or arm
added 295
 F or 38 C) or suspected fever; rash; white patches on lips, tongue or mouth (other than throat); blisters in the mouth; swollen or 'bull' neck; hoarseness or lost voice or ear pain
added 300
No: new confusion or inability to stay alert and awake; currently feeling like you are going to collapse every time you stand or sit up; purple or red rash/blotches that stay when pressed by a glass (purpuric rash) or new neck pain and difficulty bending the neck
added 301
COMBIVENT [Ipratropium-Albuterol] 18-103 mcg/Actuation Aerosol

added 428
 No: inability to speak or make normal sounds; new confusion or inability to stay alert and awake; sudden swelling of the lips, tongue, or mouth; currently struggling to breathe, even while inactive or resting; abrupt onset of breathing problems that came over the course of a minute or two; sudden onset of cough, choking or gagging after inhaling something into your airway; pain, pressure or tightness in the chest, jaw or arm; change in heart rate or rhythm or palpitations or noisy, wheezy or raspy breathing that does not clear with coughing
added 435
 No: new confusion or inability to stay alert and awake; new shortness of breath; new wheezing or chest tightness; sudden swelling of the lips, tongue, or mouth; sudden onset of trouble swallowing or purple or red rash/blotches that stay when pressed by a glass (purpural rash)
added 449
 No: other symptoms with chest pain nausea or vomiting present; other symptoms with chest pain new shortness of breath; other symptoms with ches

The individual demonstrates emerging problem solving skills for complex problem solving tasks.
added 643
 No: currently struggling to breathe, even while inactive or resting; currently feeling faint or passing out or new confusion or inability to stay alert and awake
added 648
F (38 C) or suspected fever; pain in the side of the back below the ribs (flank pain) or nausea or vomiting present
added 655
 No: inability to speak or make normal sounds; new confusion or inability to stay alert and awake; complete inability to swallow; sudden onset of coughing, choking or gagging due to inhaling something into the airway; pain, pressure or tightness in the chest, arm or shoulder, jaw, or neck; large amounts of pink or white frothy sputum or noisy, wheezy or raspy breathing that doesn't clear with coughing
added 663
 No: fever present (greater than or equal to 100.4 F or 38 C) or suspected fever; headache; very itchy eyes, nose, roof of mouth, or throat; hoarseness or lost voice; bad breath or 

Sim                                     
           mean        sum       min       max count
Topic                                               
0      0.194976  23.202154 -0.022020  0.470108   119
1      0.376012  51.889599 -0.027651  1.000000   138
2      0.176898  12.559768  0.012925  0.503801    71
3      0.217804  44.432102 -0.029964  0.449359   204
4      0.215265  32.074436  0.002171  0.516856   149
5      0.201726  13.919076 -0.016507  0.546754    69

```
Sim
	mean 	sum 	min 	max 	count
Topic 					
0 	0.194976 	23.202154 	-0.022020 	0.470108 	119
1 	0.376012 	51.889599 	-0.027651 	1.000000 	138
2 	0.176898 	12.559768 	0.012925 	0.503801 	71
3 	0.217804 	44.432102 	-0.029964 	0.449359 	204
4 	0.215265 	32.074436 	0.002171 	0.516856 	149
5 	0.201726 	13.919076 	-0.016507 	0.546754 	69

```

Output of the average of all the 138 sentences of Topic 1 compared with all the cases **Goes** with the hypothesis

The average is .38 which is highest 

In [45]:
getTopicScore(2,p_emb)

added 11
 The risks and benefits of the procedure were discussed, and the patient consented to this procedure.
added 31
Albuterol [PROVENTIL/VENTOLIN] 90 mcg/Act HFA Aerosol 2 puffs by inhalation every 4 hours as needed.
added 32
Albuterol [PROVENTIL/VENTOLIN] 90 mcg/Act HFA Aerosol 2 puffs by inhalation every 6 hours as needed.
added 41
 Discussed the risks, benefits, alternatives, advance directives, and the necessity of other members of the healthcare team participating in the procedure.
added 42
 Discussed the risks, benefits, alternatives, Advance Directives, and the necessity of other members of the healthcare team participating in the procedure.
added 46
 Discussed with patient the necessity of other members of the healthcare team, both male and female, participating in the procedure if needed.
added 49
 Nurse visit ten minutes, over half of which was spent in counseling and point-of-care testing.
added 50
 Discussed advance directives and the necessity of other members of the h

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Patient has not returned to the Travel and Tropical Medicine Clinic for the #2 hepatitis A vaccination despite having been sent two reminders.
added 124
Recheck if not resolved in one week or recheck if breathing or swallowing problems or worsening symptoms occur.
added 134
 Discussed advance directives and the necessity of other members of the healthcare team, both male and female, participating in the procedure.
added 153
Patient was identified using two identifiers and permission was obtained from the patient to perform a fingerstick INR.
added 155
 Discussed advance directives and the necessity of other members of the healthcare team, both male and female, participating in the procedure.
added 181
 Discussed advance directives and the necessity of other members of the healthcare team, both male and female, participating in the procedure.
added 184
 Discussed the risks, benefits, alternatives, advance directives, and the necessity of other members of the healthcare team participatin

Sim                                     
           mean        sum       min       max count
Topic                                               
0      0.307324  36.571613  0.016920  0.648725   119
1      0.176898  24.411943 -0.028023  0.560037   138
2      0.297501  21.122541  0.027818  1.000000    71
3      0.273393  55.772247  0.004143  0.554788   204
4      0.276909  41.259373 -0.014807  0.550348   149
5      0.264528  18.252464  0.026920  0.622063    69

```

 	Sim
	mean 	sum 	min 	max 	count
Topic 					
0 	0.307324 	36.571613 	0.016920 	0.648725 	119
1 	0.176898 	24.411943 	-0.028023 	0.560037 	138
2 	0.297501 	21.122541 	0.027818 	1.000000 	71
3 	0.273393 	55.772247 	0.004143 	0.554788 	204
4 	0.276909 	41.259373 	-0.014807 	0.550348 	149
5 	0.264528 	18.252464 	0.026920 	0.622063 	69

```

Output of the average of all the 71 sentences Topic 2 compared with all the cases **Doesn't Go** with the hypothesis

The average is .297 which is 2nd highest. Topic 0 is higher in value 

In [46]:
getTopicScore(3,p_emb)

added 9
 Negative gastrointestinal review of systems, Historian denies abdominal pain, nausea, vomiting.
added 14
 Explained diagnosis and treatment plan; patient expressed understanding of the content, Patient was given a copy of this note
added 17
 Identified Illness as a learning need, Patients primary language is English, No Barriers to learning were identified, No interventions were used to address Barriers to Learning, Teaching methods used included Printed Patient Instructions, Verbal Instructions, Patient demonstrated and verbalized an understanding of discharge teaching.
added 18
 Identified Illness as a learning need, No Barriers to learning were identified, No interventions were used to address Barriers to Learning, Teaching methods used included Printed Patient Instructions, Patient verbalized an understanding of discharge teaching.
added 21
 Identified Illness as a learning need, No Barriers to learning were identified, No interventions were used to address Barriers to Lea

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Explained diagnosis and treatment plan; patient/child/caregiver expressed understanding of the content..
added 70
 Ready to learn, no apparent learning barriers were identified in the family/caregiver; learning preferences include listening.
added 71
Education provided on diagnosis and treatment plan using verbal instruction; patient expressed acceptance of this and understanding of the content.
added 76
 HIPAA authorization reviewed, dated, and signed by participant and filed in the Mayo medical record.
added 80
 By dismissal she was tolerating a general diet, ambulating, afebrile, pain controlled, and having normal bowel and bladder function.
added 86
There were no apparent barriers to learning or understanding and all questions were answered.
added 89
Relayed these directions to the patient by telephone who reports an adequate understanding of the plan and denies noting any signs of bleeding or recurrent stroke-like symptoms.
added 90
 Identified Illness as a learning need, No Barri

 The above was discussed with the patient, and she voiced understanding of the content and plan.
added 212
 Patient ready to learn, no apparent learning barriers were identified; learning preferences include listening.
added 224
 Ready to learn, no apparent learning barriers were identified; learning preferences including listening.
added 227
 Identified Injury as a learning need, Patients primary language is English, No Barriers to learning were identified, No interventions were used to address Barriers to Learning, Teaching methods used included Printed Patient Instructions, Patient verbalized an understanding of discharge teaching.
added 231
 The patient had a chance to have any questions about this procedure answered, understand(s) and wish(es) to proceed.
added 233
The patient understands the information and questions answered; the patient wishes to proceed with the biopsy.
added 234
The diagnosis and treatment plans were explained and the patient expressed understanding of the co

 Negative gastrointestinal review of systems, Historian denies abdominal pain, diarrhea, nausea, vomiting.
added 405
 Identified Illness as a learning need, No Barriers to learning were identified, No interventions were used to address Barriers to Learning, Teaching methods used included Printed Patient Instructions.
added 419
The diagnosis and treatment plans were explained and the patient expressed understanding of the content.
added 421
Explained diagnosis and treatment plan; patient/parents expressed understanding of the content.
added 422
 Sleep Plan of Care Follow-up: patient was provided with a nurses business card and encouraged to phone our Sleep Center in the event problems or questions regarding treatment plan occur, follow-up visit with nurse in one year
added 424
 Caller verbalizes understanding of protocol advice, and understanding of call back instructions
added 427
 Identified Illness as a learning need, No Barriers to learning were identified, Teaching methods used inc

added 625
 Patient discharged to home, carried, family driving, accompanied by parent, Above person(s) verbalized understanding of discharge instructions and follow-up care.
added 633
 Identified Illness as a learning need, Patients primary language is English, No Barriers to learning were identified, No interventions were used to address Barriers to Learning, Teaching methods used included Printed Patient Instructions, Patient verbalized an understanding of discharge teaching.
added 636
Patient Education:  Ready to learn; no apparent learning barriers were identified; learning preferences include listening.
added 646
 Identified Injury as a learning need, Patients primary language is English, No Barriers to learning were identified, No interventions were used to address Barriers to Learning, Teaching methods used included Printed Patient Instructions, Patient verbalized an understanding of discharge teaching.
added 653
Result is: Supra Therapeutic; result and dosage information phoned

Sim                                     
           mean        sum       min       max count
Topic                                               
0      0.377580  44.932041  0.051793  0.634533   119
1      0.217804  30.057013 -0.003120  0.473373   138
2      0.273393  19.410931  0.071450  0.474550    71
3      0.484400  98.817673  0.051173  1.000000   204
4      0.354159  52.769711  0.053555  0.637730   149
5      0.330672  22.816378  0.093018  0.611079    69

```

Sim
	mean 	sum 	min 	max 	count
Topic 					
0 	0.377580 	44.932037 	0.051793 	0.634533 	119
1 	0.217804 	30.057013 	-0.003120 	0.473373 	138
2 	0.273393 	19.410931 	0.071450 	0.474550 	71
3 	0.484400 	98.817665 	0.051173 	1.000000 	204
4 	0.354159 	52.769711 	0.053555 	0.637730 	149
5 	0.330672 	22.816378 	0.093018 	0.611079 	69

```

Output of the average of all the 204 sentences of Topic 3 compared with all the cases **Goes** with the hypothesis

The average is .48 which is highest 

In [47]:
getTopicScore(4,p_emb)

added 1
 Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears comfortable, Patient cooperative, alert, Skin warm.
added 2
 Peripheral IV site, established in the right forearm, using an 18 gauge catheter, in one attempt.
added 15
 Patient arrives, via stretcher, via Emergency Medical Services, Unsteady gait, Lift to cart, History obtained from patient, Patient appears comfortable, Patient cooperative, alert, Oriented to person, place and time.
added 33
 Patient arrives, via hospital wheelchair, History obtained from patient, Patient appears, uncomfortable, Patient cooperative, alert, Oriented to person, place and time.
added 40
 Consultation:  consultant not available, patient did not wish to wait, patient scheduled for consultation at next visit.
added 47
Patient is here for the following immunization(s):  Inactivated Influenza Virus Vaccination; Tetanus and Diphtheria and Acellular Pertussis (Tdap) Vaccine
added 52
 Educational literature, developme

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


 The patient has had a chance to have their questions about this procedure answered and wishes to proceed., Procedural pause conducted to verify: correct patient identity, procedure to be performed and as applicable, correct side and site, correct patient position, and availability of implants, special equipment or special requirements
added 56
 Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears, anxious, Patient cooperative, alert, Oriented to person, place and time.
added 66
 Patient needs assistance with the following instrumental activities of daily  living: meal preparation, medication administration, housekeeping, shopping, managing finances, transportation use (drive car/use taxi/bus).
added 68
Patient arrives, via hospital wheelchair, History obtained from patient, Patient appears, in distress due to pain, Patient cooperative, alert, Oriented to person, place and time.
added 73
 Patient arrives, via Emergency Medical Services, Unsteady gait,

added 311
 Procedural pause conducted to verify the following: correct patient identity, procedure to be performed and as applicable, correct side and site, correct patient position, and availability of implants, special equipment, or special requirements.
added 314
Patient is here for the following immunization(s):  Inactivated Influenza Virus Vaccination; Tetanus and Diphtheria and Acellular Pertussis (Tdap) Vaccine
added 320
 Patient's age is 9 years of age to less than 18 years old: Administer Inactivated Influenza Virus Vaccine (Fluzone) 0.5 mL, intramuscular.
added 326
 Patient arrives, via stretcher, via Emergency Medical Services, Unsteady gait, Lift to cart, History obtained from patient, Patient appears comfortable, Patient cooperative, alert, Oriented to person, place and time.
added 328
 Procedural pause conducted prior to each stage of procedure to verify: correct patient identity, procedure to be performed and as applicable, correct side and site, special equipment or spe

added 553
Goals: Patient will demonstrate a 50% decrease in pain in 4-6 sessions for increased participation in life or work activities.
added 554
 Respiratory assessment findings include respiratory effort easy, Respirations regular, Conversing normally.
added 568
The patient does not have hypersensitivity to local anesthetic or depot steroid preparations.
added 571
 Peripheral IV site, established in the right forearm, using a 20 gauge catheter, in one attempt.
added 575
 Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears comfortable, Patient cooperative, alert.
added 582
Patient will demonstrate and/or verbalize understanding of home exercise program in 1 sessions for improved self management of the condition.
added 585
Patient is here for the following immunization(s):  Live Intranasal Influenza Virus Vaccination
added 592
The patient will meet individually with the group provider nearing completion of LEARN and prior to bariatric surgery.
added

Sim                                     
           mean        sum       min       max count
Topic                                               
0      0.362129  43.093319  0.052916  0.635205   119
1      0.215265  29.706526 -0.000029  0.518147   138
2      0.276909  19.660505  0.046798  0.514246    71
3      0.354159  72.248466  0.032752  0.595916   204
4      0.386240  57.549824  0.013457  1.000000   149
5      0.307874  21.243305  0.083924  0.545824    69

```

Sim
	mean 	sum 	min 	max 	count
Topic 					
0 	0.362129 	43.093319 	0.052916 	0.635205 	119
1 	0.215265 	29.706524 	-0.000029 	0.518147 	138
2 	0.276909 	19.660507 	0.046798 	0.514246 	71
3 	0.354159 	72.248466 	0.032752 	0.595916 	204
4 	0.386240 	57.549824 	0.013457 	1.000000 	149
5 	0.307874 	21.243305 	0.083924 	0.545824 	69

```

Output of the average of all the 149 sentences of Topic 4 compared with all the cases **Goes** with the hypothesis

The average is .38 which is highest 

In [48]:
getTopicScore(5,p_emb)

added 4
Spent 15 minutes with the patient and greater than 50% of this time was spent counseling the patient regarding diagnosis and available treatment options.
added 8
 Discussed the necessity of other members of the healthcare team, both male and female, participating in the procedure.
added 25
 Discussed:  patient's current frequency of brushing and flossing, need for regular flossing.
added 51
The patient remained focused on group tasks and displayed interest in others and in the group process.
added 63
No: cloudy or foul-smelling urine; blood in the urine; new daytime wetting; new bedwetting; changes in urine color; foamy urine or vaginal discharge or other genital problem
added 74
 Discussed the risks, benefits, alternatives, and the necessity of other members of the healthcare team participating in the procedure.
added 75
This information will be reviewed by the provider and the patient/caregiver will be contacted with any recommendations.
added 78
 Learns best by: verbal instr

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


added 149
 During or after exercise, patient denies:  rash or hives, vertigo, syncope episode, chest pain, shortness of breath, wheeze, cough, palpitations, heat-related illness, excessive fatigue, headaches.
added 157
Explained diagnosis and treatment plan; patient/child/caregiver expressed understanding of the content.Caller agrees to treatment recommendations given over the phone
added 167
 GI: no nausea, vomiting, diarrhea, constipation, hematochezia, melena, or other problems noted.
added 191
This information will be reviewed by the provider and the patient/caregiver will be contacted with any recommendations.
added 196
 Goals/outcomes:  Patient will be instructed in home program, demonstrate understanding, and state ability to continue independently.
added 221
 Discussed acne and treatment in detail along with consent with risks, alternatives, and benefits.
added 229
 Cyanocobalamin [VITAMIN B12] 1,000 mcg/mL solution 1,000 micrograms intramuscular as directed by prescriber.
adde

Sim                                     
           mean        sum       min       max count
Topic                                               
0      0.341318  40.616856  0.017080  0.608576   119
1      0.201726  27.838152 -0.022102  0.493165   138
2      0.264528  18.781523  0.040569  0.581541    71
3      0.330672  67.457115  0.025114  0.593672   204
4      0.307874  45.873226 -0.000321  0.568237   149
5      0.307110  21.190582  0.043178  1.000000    69

```
Sim
	mean 	sum 	min 	max 	count
Topic 					
0 	0.341318 	40.616859 	0.017080 	0.608576 	119
1 	0.201726 	27.838152 	-0.022102 	0.493165 	138
2 	0.264529 	18.781525 	0.040569 	0.581541 	71
3 	0.330672 	67.457123 	0.025114 	0.593672 	204
4 	0.307874 	45.873230 	-0.000321 	0.568237 	149
5 	0.307110 	21.190582 	0.043178 	1.000000 	69


```

Output of the average of all the 69 sentences Topic 5 compared with all the cases **Doesn't Go** with the hypothesis

The average is .30 which is 3rd highest. Topic 0 is higher in value 

**Check the topic identified works similar for Column 2 of the Clinical STS dataset**

In [49]:
ClinicalTexts[1].tolist()[0:5]

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[' Insulin NPH Human [NOVOLIN N] 100 unit/mL suspension 63-76 units subcutaneous as directed by prescriber.',
 ' Complex assessment performed, Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears, anxious, in distress due to pain, Patient cooperative, alert, Oriented to person, place and time.',
 ' Peripheral IV site, present prior to arrival, established in the right hand, using a 20 gauge catheter.',
 ' No: new confusion or inability to stay alert and awake; any chest pain or discomfort; sudden, CURRENTand excruciating ripping or stabbing pain characteristic of a dissecting aortic aneurysm; newly painful or blue toes on one side; currently struggling to breathe, even while inactive or resting; currently feeling like you are going to collapse every time you stand (sit); vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools; heavy vaginal bleeding or purple or red rash/blotche

Preprocess the text

In [50]:
corpus1 = ClinicalTexts[1].tolist()
# clean data stored in a new list
clean_corpus1 = [clean(doc).split() for doc in corpus1]   

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Creating the Document term Matrix

Now, the next step is to convert the corpus (the list of documents) into a document-term Matrix using the dictionary that we had prepared above. (The vectorizer used here is the Bag of Words).	

In [51]:
# Converting list of documents (corpus) into Document Term Matrix using the dictionary 
doc_term_matrix1 = [dict_.doc2bow(i) for i in clean_corpus1]
doc_term_matrix1[0:2]

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (886, 1)],
 [(11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 4),
  (22, 1),
  (74, 1),
  (102, 1),
  (130, 1),
  (169, 1),
  (170, 1),
  (171, 1),
  (418, 1),
  (425, 1),
  (467, 1),
  (468, 1),
  (486, 1)]]

In [52]:
# printing the topic associations with the documents
import operator
lstTopicsCorpus1 =[]

count = 0
for i in ldamodel[doc_term_matrix1]:
    print("doc : ",count, i)
    maxTopic = max(i,key=operator.itemgetter(1))[0]
    lstTopicsCorpus1.append(maxTopic)
    count += 1

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


doc :  0 [(0, 0.93563145), (1, 0.012823553), (2, 0.013020256), (3, 0.012834175), (4, 0.012834128), (5, 0.012856375)]
doc :  1 [(4, 0.96652853)]
doc :  2 [(0, 0.012849201), (1, 0.31842217), (2, 0.0129478555), (3, 0.012834589), (4, 0.63011885), (5, 0.012827362)]
doc :  3 [(0, 0.19562118), (1, 0.7943947)]
doc :  4 [(0, 0.016732955), (1, 0.01670794), (2, 0.9162441), (3, 0.016700462), (4, 0.016757075), (5, 0.0168575)]
doc :  5 [(0, 0.010474375), (1, 0.94778556), (2, 0.010462256), (3, 0.010420185), (4, 0.010434499), (5, 0.010423123)]
doc :  6 [(1, 0.9739019)]
doc :  7 [(0, 0.79516894), (1, 0.016795598), (2, 0.016838118), (3, 0.016758312), (4, 0.016869975), (5, 0.1375691)]
doc :  8 [(0, 0.37235224), (1, 0.0416924), (2, 0.04169454), (3, 0.4608271), (4, 0.041693106), (5, 0.04174058)]
doc :  9 [(0, 0.020853035), (1, 0.02106764), (2, 0.020847524), (3, 0.222766), (4, 0.020909626), (5, 0.69355613)]
doc :  10 [(0, 0.8041765), (1, 0.13476768), (2, 0.015254756), (3, 0.015342176), (4, 0.015235399), (5,

doc :  549 [(0, 0.95589054)]
doc :  550 [(0, 0.84957373), (1, 0.11320459)]
doc :  551 [(0, 0.011149182), (1, 0.0111625055), (2, 0.011133892), (3, 0.94430286), (4, 0.011139046), (5, 0.011112499)]
doc :  552 [(0, 0.015276627), (1, 0.0151778385), (2, 0.015306612), (3, 0.025781661), (4, 0.91311586), (5, 0.015341434)]
doc :  553 [(0, 0.011949586), (1, 0.012029217), (2, 0.011971097), (3, 0.0119526675), (4, 0.41448337), (5, 0.5376141)]
doc :  554 [(4, 0.95605445)]
doc :  555 [(0, 0.027873553), (1, 0.488473), (2, 0.028245283), (3, 0.027865862), (4, 0.1998736), (5, 0.22766872)]
doc :  556 [(0, 0.39109167), (1, 0.015187349), (2, 0.015252704), (3, 0.22263306), (4, 0.34056285), (5, 0.015272334)]
doc :  557 [(0, 0.016979584), (1, 0.016720975), (2, 0.1192936), (3, 0.65065026), (4, 0.016980689), (5, 0.17937483)]
doc :  558 [(0, 0.6943382), (1, 0.086571895), (2, 0.011173296), (3, 0.1855053), (4, 0.011241959), (5, 0.011169319)]
doc :  559 [(0, 0.016867066), (1, 0.01672993), (2, 0.9161057), (3, 0.016760

In [57]:
df_Topic_both = pd.DataFrame(
    {'Text1': corpus,
     'Text2' : corpus1,
     'Cleaned_Text1': clean_corpus,
     'Cleaned_Text2': clean_corpus1,
     'Topic1': lstTopicsCorpus,
     'Topic2': lstTopicsCorpus1,
     'human_score':ClinicalTexts[2].tolist()
    })

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [58]:
df_Topic_both

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Text1,Text2,Cleaned_Text1,Cleaned_Text2,Topic1,Topic2,human_score
0,Insulin NPH Human [NOVOLIN N] 100 unit/mL suspension subcutaneous as directed by prescriber.,Insulin NPH Human [NOVOLIN N] 100 unit/mL suspension 63-76 units subcutaneous as directed by prescriber.,"[insulin, nph, human, novolin, n, 100, unitml, suspension, subcutaneous, directed, prescriber]","[insulin, nph, human, novolin, n, 100, unitml, suspension, 6376, unit, subcutaneous, directed, prescriber]",0,0,3.50
1,"Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears comfortable, Patient cooperative, alert, Skin warm.","Complex assessment performed, Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears, anxious, in distress due to pain, Patient cooperative, alert, Oriented to person, place and time.","[patient, arrives, ambulatory, gait, steady, history, obtained, patient, patient, appears, comfortable, patient, cooperative, alert, skin, warm]","[complex, assessment, performed, patient, arrives, ambulatory, gait, steady, history, obtained, patient, patient, appears, anxious, distress, due, pain, patient, cooperative, alert, oriented, person, place, time]",4,4,2.50
2,"Peripheral IV site, established in the right forearm, using an 18 gauge catheter, in one attempt.","Peripheral IV site, present prior to arrival, established in the right hand, using a 20 gauge catheter.","[peripheral, iv, site, established, right, forearm, using, 18, gauge, catheter, one, attempt]","[peripheral, iv, site, present, prior, arrival, established, right, hand, using, 20, gauge, catheter]",4,4,3.45
3,"No: new confusion or inability to stay alert and awake; currently struggling to breathe, even while inactive or resting; currently feeling like you are going to collapse every time you stand (sit); vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools; heavy vaginal bleeding or purple or red rash/blotches that stay when pressed by a glass (purpural rash)","No: new confusion or inability to stay alert and awake; any chest pain or discomfort; sudden, CURRENTand excruciating ripping or stabbing pain characteristic of a dissecting aortic aneurysm; newly painful or blue toes on one side; currently struggling to breathe, even while inactive or resting; currently feeling like you are going to collapse every time you stand (sit); vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools; heavy vaginal bleeding or purple or red rash/blotches that stay when pressed by a glass (purpural rash)","[no, new, confusion, inability, stay, alert, awake, currently, struggling, breathe, even, inactive, resting, currently, feeling, like, going, collapse, every, time, stand, sit, vomit, look, like, ground, coffee, vomiting, blood, uncontrollable, continuous, rectal, bleeding, black, sticky, tarlike, stool, heavy, vaginal, bleeding, purple, red, rashblotches, stay, pressed, glass, purpural, rash]","[no, new, confusion, inability, stay, alert, awake, chest, pain, discomfort, sudden, currentand, excruciating, ripping, stabbing, pain, characteristic, dissecting, aortic, aneurysm, newly, painful, blue, toe, one, side, currently, struggling, breathe, even, inactive, resting, currently, feeling, like, going, collapse, every, time, stand, sit, vomit, look, like, ground, coffee, vomiting, blood, uncontrollable, continuous, rectal, bleeding, black, sticky, tarlike, stool, heavy, vaginal, bleeding, purple, red, rashblotches, stay, pressed, glass, purpural, rash]",1,1,4.00
4,Spent 15 minutes with the patient and greater than 50% of this time was spent counseling the patient regarding diagnosis and available treatment options.,"Nurse visit ten minutes, over half of which was spent in counseling and point-of-care testing.","[spent, 15, minute, patient, greater, 50, time, spent, counseling, patient, regarding, diag

In [69]:
df_Topic_both.loc[df_Topic_both['Topic1'] == df_Topic_both['Topic2']][['Text1','Text2','human_score','Topic1','Topic2']]

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Text1,Text2,human_score,Topic1,Topic2
0,Insulin NPH Human [NOVOLIN N] 100 unit/mL suspension subcutaneous as directed by prescriber.,Insulin NPH Human [NOVOLIN N] 100 unit/mL suspension 63-76 units subcutaneous as directed by prescriber.,3.50,0,0
1,"Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears comfortable, Patient cooperative, alert, Skin warm.","Complex assessment performed, Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears, anxious, in distress due to pain, Patient cooperative, alert, Oriented to person, place and time.",2.50,4,4
2,"Peripheral IV site, established in the right forearm, using an 18 gauge catheter, in one attempt.","Peripheral IV site, present prior to arrival, established in the right hand, using a 20 gauge catheter.",3.45,4,4
3,"No: new confusion or inability to stay alert and awake; currently struggling to breathe, even while inactive or resting; currently feeling like you are going to collapse every time you stand (sit); vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools; heavy vaginal bleeding or purple or red rash/blotches that stay when pressed by a glass (purpural rash)","No: new confusion or inability to stay alert and awake; any chest pain or discomfort; sudden, CURRENTand excruciating ripping or stabbing pain characteristic of a dissecting aortic aneurysm; newly painful or blue toes on one side; currently struggling to breathe, even while inactive or resting; currently feeling like you are going to collapse every time you stand (sit); vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools; heavy vaginal bleeding or purple or red rash/blotches that stay when pressed by a glass (purpural rash)",4.00,1,1
5,No: new confusion or inability to stay alert and awake; newly painful neck with difficulty bending the neck; paralyzed face muscles or severe dizziness,No: new confusion or inability to stay alert and awake or currently feeling like she is going to collapse every time she stands or sits up,2.00,1,1
...,...,...,...,...,...
744,"Patient discharged to home, ambulating without assistance, family driving, accompanied by other family member, Above person(s) verbalized understanding of discharge instructions and follow-up care.","Patient discharged to home, ambulating without assistance, driving self, unaccompanied, Discharge instructions given to patient, Prescriptions given and instructions on side effects given, Above person(s) verbalized understanding of discharge instructions and follow-up care.",3.00,0,0
745,Discussed the necessity of other members of the team participating in the procedure with the patient.,"We discussed the risks, the goals, the alternatives, and the necessity of other members of the healthcare team who would be participating in the procedures with the patient.",3.00,0,0
746,"Barriers to learning: (cultural, religious/spiritual, motivational, physical/cognitive, language, emotional); no apparent barriers to learning present, ready to learn.","Barriers assessed (Cultural, Religious/Spiritual, Motivational, Physical/Cognitive, Language, Emotional).",2.75,3,3
747,"Unable to assess if there is a history or concern, or exposure to physical, emotional, sexual, financial abuse, neglect or domestic violence, Comments: family present.","Unable to assess if there is a history or concern, or exposure to physical, emotional, sexual, financial abuse, neglect or domestic violence, Comments: child.",3.90,2,2


In [70]:
df_Topic_both.loc[df_Topic_both['Topic1'] != df_Topic_both['Topic2']][['Text1','Text2','human_score','Topic1','Topic2']]

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Text1,Text2,human_score,Topic1,Topic2
4,Spent 15 minutes with the patient and greater than 50% of this time was spent counseling the patient regarding diagnosis and available treatment options.,"Nurse visit ten minutes, over half of which was spent in counseling and point-of-care testing.",3.00,5,2
8,"Discussed the necessity of other members of the healthcare team, both male and female, participating in the procedure.",This report represents the interpretation of the original data and tracings that are stored in the electronic records of the Esophageal Laboratory.,0.00,5,3
9,"Negative gastrointestinal review of systems, Historian denies abdominal pain, nausea, vomiting.","Negative genitourinary review of systems, Historian denies dysuria, hematuria, urine output changes.",2.50,3,5
11,"The risks and benefits of the procedure were discussed, and the patient consented to this procedure.","The content of this note has been reproduced, signed by an authorized physician in the space above, and mailed to the patient's parents, the patient's home care company.",0.50,2,0
16,"Dimension 3 Emotional, Behavioral or Cognitive Conditions and Complications: The patient received a Risk Score of 1. Interference with Addiction Recovery Emotional concerns related to negative consequences and effects of addiction; patient is able to view these as part of addiction and recovery.","Dimension 3 Emotional, Behavioral or Cognitive Conditions and Complications: The patient received a Risk Score of 2. He/She reports or demonstrates symptoms of depression",2.50,0,4
...,...,...,...,...,...
702,"Discussed the risks, benefits, alternatives, and the necessity of other members of the healthcare team participating in the procedure.","Discussed risks, goals, alternatives, and necessity of other members of the team participating in the procedure with patient.",4.75,5,0
703,"Discussed with the patient the necessity of other members of the healthcare team, both male and female, participating in the procedure if needed.","Discussed risks, goals, alternatives, and the necessity of other members of the surgical team participating in the procedure with patient and others present.",3.40,2,0
706,"Discussed risks, goals, alternatives, advanced directives and the necessity of other members of the healthcare team participating in the procedure.","We discussed risks, goals, and alternatives in detail, hopeful results, and possible complications.",3.25,0,5
708,No: new deformity other than swelling; a grating or crackling sound or sensation on movement; bruising or cut or break in the skin,No: extremely dry mouth; dizzy when sitting or standing; new wheezing or severe or persistent vomiting,0.00,2,1
